In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()

db = client.restaurants

for rest in db.restaurants.find({"name":"Burger King"}):
    print("BK placed in {} {}".format(rest["borough"], rest["address"]["street"]))


In [2]:
from pandas.io.json import json_normalize

def prepareForTableau(bks):
    locations = json_normalize(bks["address"])
    gelocs = locations.apply(lambda e: [e["coord"][0],e["coord"][1]], result_type="expand", axis=1)
    clean_df = pd.concat([bks["restaurant_id"],locations[["street","zipcode"]],gelocs], axis=1)
    clean_df.rename({1:"lat",0:"long"}, axis=1, inplace=True)
    return clean_df

In [3]:
bks = pd.DataFrame(db.restaurants.find({"name":"Burger King"}))
bks.head()

clean_df = prepareForTableau(bks)
display(clean_df.head())


KeyError: 'address'

In [ ]:
clean_df.to_json("bk_rests.json",orient="records")

In [4]:

nearLocation = {
    "lng": 37.793568,
    "lat": -122.404621
}

bks = pd.DataFrame(db.companies.find({
    "address.coord": {
     "$near": {
       "$geometry": {
          "type": "Point" ,
          "coordinates": [ nearLocation["lat"] , nearLocation["lng"] ]
       },
       "$maxDistance": 2000, # In meters
     }
   }
}))
bks.head()



""


In [ ]:
clean_df = prepareForTableau(bks)
display(clean_df.head())
clean_df.to_json("bk_rests_filtered.json",orient="records")